# Lecture 48: Global Activation Pooling

### Dataset used:- [ALL-IDB:Acute Lymphoblastic Leukemia Image Database for Image Processing](https://homes.di.unimi.it/scotti/all/)
Follow the instructions provided in the linked website to download the dataset. After downloading, extract the files to the current directory (same folder as your codes).

In [ ]:
%matplotlib inline
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from torch.autograd import Variable
from torch.utils.data import TensorDataset,DataLoader
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time
import os
import copy

### Load Data

In [ ]:
Datapath = 'ALL_IDB2/img/'
listing = os.listdir(Datapath) 

In [ ]:
# ALL_IDB2 dataset has 260 images in total
TrainImages = torch.FloatTensor(200,3,224,224)
TrainLabels = torch.LongTensor(200)
TestImages = torch.FloatTensor(60,3,224,224)
TestLabels = torch.LongTensor(60)

# First 200 images are used for training and the remaining 60 for testing
img_no = 0
for file in listing:
    im=Image.open(Datapath + file)
    im = im.resize((224,224))
    im = np.array(im)   
    if img_no < 200:
        TrainImages[img_no] = torch.from_numpy(im).transpose(0,2).unsqueeze(0)
        TrainLabels[img_no] = int(listing[img_no][6:7])
    else:
        TestImages[img_no - 200] = torch.from_numpy(im).transpose(0,2).unsqueeze(0)
        TestLabels[img_no - 200] = int(listing[img_no][6:7])
    img_no = img_no + 1

In [ ]:
print(TrainImages.size())
print(TrainLabels.size())
print(TestImages.size())
print(TestLabels.size())

In [ ]:
# Creating pytorch dataset
trainDataset = TensorDataset(TrainImages, TrainLabels)
testDataset = TensorDataset(TestImages, TestLabels)
# Creating dataloader
BatchSize = 32
trainLoader = DataLoader(trainDataset, batch_size=BatchSize, shuffle=True,num_workers=4, pin_memory=True)
testLoader = DataLoader(testDataset, batch_size=BatchSize, shuffle=True,num_workers=4, pin_memory=True)

In [ ]:
# Check availability of GPU
use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')

### Initialize the network

In [ ]:
# ResNet18
net = models.resnet18(pretrained=True)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 2)
print(net)
if use_gpu:
    net = net.cuda()

### Define loss function and optimizer

In [ ]:
criterion = nn.NLLLoss() # Negative Log-Likelihood
optimizer = optim.SGD(net.parameters(), lr=1e-3 , momentum=0.9) # Stochastic gradient descent

### Train the network

In [ ]:
iterations = 15
trainLoss = []
trainAcc = []
testLoss = []
testAcc = []
start = time.time()

for epoch in range(iterations):
    epochStart = time.time()
    runningLoss = 0   
    runningCorr = 0
    net.train(True) # For training
    for data in trainLoader:
        inputs,labels = data
        # Wrap them in Variable
        if use_gpu:
            inputs, labels = Variable(inputs.float().cuda()), \
                Variable(labels.long().cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labelslong())          
       
        inputs = inputs/255.0
        # Feed-forward input data through the network
        outputs = net(inputs)
        # Compute loss/error
        loss = criterion(F.log_softmax(outputs), labels) 
        
        _, predicted = torch.max(outputs.data, 1)
        # Initialize gradients to zero
        optimizer.zero_grad()                  
        # Backpropagate loss and compute gradients
        loss.backward()
        # Update the network parameters
        optimizer.step()
        # Accumulate loss per batch
        runningLoss += loss.data[0]  
        # Accumuate correct predictions per batch
        runningCorr += (predicted == labels.data).sum()
    avgTrainLoss = runningLoss/200.0
    avgTrainAcc = runningCorr/200.0
    trainLoss.append(avgTrainLoss)
    trainAcc.append(avgTrainAcc)
    
    # Evaluating performance on test set for each epoch
    net.train(False) # For testing
    test_runningCorr = 0
    test_runningLoss = 0
    for data in testLoader:
        inputs,labels = data
        # Wrap them in Variable
        if use_gpu:
            inputs, labels = Variable(inputs.float().cuda()), \
                Variable(labels.long().cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labelslong())  
        inputs = inputs/255
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)    
         # Compute loss/error
        loss = criterion(F.log_softmax(outputs), labels)      
        # Accumulate loss per batch
        test_runningLoss += loss.data[0]  
        # Accumuate correct predictions per batch
        test_runningCorr += (predicted == labels.data).sum()
    avgTestLoss = test_runningLoss/60.0
    avgTestAcc = test_runningCorr/60.0
    testAcc.append(avgTestAcc)
    testLoss.append(avgTestLoss)
        
    # Plotting Loss vs Epochs
    fig1 = plt.figure(1)        
    plt.plot(range(epoch+1),trainLoss,'r--',label='train')        
    plt.plot(range(epoch+1),testLoss,'g--',label='test')        
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')    
    # Plotting testing accuracy vs Epochs
    fig2 = plt.figure(2)        
    plt.plot(range(epoch+1),trainAcc,'r-',label='train') 
    plt.plot(range(epoch+1),testAcc,'g-',label='test')        
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')    
    
    epochEnd = time.time()-epochStart
    print('At Iteration: {:.0f} /{:.0f}  ;  Training Loss: {:.6f} ; Training Acc: {:.3f} ; Time consumed: {:.0f}m {:.0f}s '\
          .format(epoch + 1,iterations,avgTrainLoss,avgTrainAcc*100,epochEnd//60,epochEnd%60))
    print('At Iteration: {:.0f} /{:.0f}  ;  Testing Loss: {:.6f} ; Testing Acc: {:.3f} ; Time consumed: {:.0f}m {:.0f}s '\
          .format(epoch + 1,iterations,avgTestLoss,avgTestAcc*100,epochEnd//60,epochEnd%60))
end = time.time()-start
print('Training completed in {:.0f}m {:.0f}s'.format(end//60,end%60))

In [ ]:
# Extracting the convolutional layers of the network
conv_net = nn.Sequential(*list(net.children())[:-2])
print(conv_net)

In [ ]:
# Copying weights of the final layer for obtaining the segmented output
weights = copy.deepcopy(net.fc.weight.data.cpu()).numpy()
print(weights.shape)

In [ ]:
# Loading one sample image for testing
testPath = 'ALL_IDB1/img/'
testImages = os.listdir(testPath)
img1 = plt.imread(testPath+testImages[0])
if use_gpu:
    testInput = Variable(torch.from_numpy(img1).transpose(0,2).transpose(1,2).unsqueeze(0)).float().cuda()
else:
    testInput = Variable(torch.from_numpy(img1).transpose(0,2).transpose(1,2).unsqueeze(0)).float()
# Feed-forward
out = conv_net(testInput)  

# Visualization
if use_gpu:
    out_np = out.squeeze(0).data.cpu().numpy()
else:
    out_np = out.squeeze(0).data.numpy()

mask1 = np.ones(out_np.shape)
for n1 in range(512):
    mask1[n1] = weights[0,n1]*mask1[n1]
outImg1 = np.sum(np.multiply(mask1,out_np),axis=0)

# Averaged activation map
plt.figure()
plt.subplot(121)
plt.imshow(np.sum(out_np,axis=0)/512,cmap='gray')
# Weighted-sum activation map
plt.subplot(122)
plt.imshow(outImg1,cmap='gray')

# Activation maps chosen at random
plt.figure()
randIdx = np.random.randint(0,511,4)
plt.subplot(141)
plt.imshow(out_np[randIdx[0]],cmap='gray')
plt.subplot(142)
plt.imshow(out_np[randIdx[1]],cmap='gray')
plt.subplot(143)
plt.imshow(out_np[randIdx[2]],cmap='gray')
plt.subplot(144)
plt.imshow(out_np[randIdx[3]],cmap='gray')

In [ ]:
plt.imshow(img1)